# Bing Grounding with Azure AI Agent


In [9]:
import os
from azure.ai.projects.models import BingGroundingTool
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
USER_INPUTS = [
    "What is today's weather in South Korea?",
    "What is the new hotels in NYC 2025?",
]

creds = DefaultAzureCredential()

project_client = AIProjectClient.from_connection_string(
    credential=creds,
    conn_str=os.getenv("AZURE_AI_AGENT_PROJECT_CONNECTION_STRING"),
)
bing_connection = project_client.connections.get(
    connection_name=os.getenv("BING_GROUNDING_CONNECTION_NAME"),
)
conn_id = bing_connection.id


# 1. Create an agent with a code interpreter on the Azure AI agent service
bing = BingGroundingTool(connection_id=conn_id)
with project_client:
    agent = project_client.agents.create_agent(
        model=os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"),
        name="my-bing-agent",
        instructions="You are a helpful assistant",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    #if you want to use existing agent, you can use the following code
    # agent = project_client.agents.get_agent(agent_id="your agent id")
    
    # Create thread for communication
    thread = project_client.agents.create_thread()
    
    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=USER_INPUTS[0],
    )
    print(f"Created message, ID: {message.id}")

    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    
    run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
    run_steps_data = run_steps['data']
    
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    #Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    
    for msg in messages['data']:
        if msg['role'] == 'assistant':
            print(f"Message ID: {msg['id']}")
            print(f"Role: {msg['role']}")
            print(f"Content: {msg['content'][0]['text']['value']}")
            print(f"Created At: {msg['created_at']}")
            print(f"Metadata: {msg['metadata']}")
            
            # Print URL information if available
            annotations = msg['content'][0]['text'].get('annotations', [])
            for annotation in annotations:
                if annotation['type'] == 'url_citation':
                    url_info = annotation['url_citation']
                    print(f"URL: {url_info['url']}")
                    print(f"Title: {url_info['title']}")
            print("-" * 50)

Created message, ID: msg_oEvhGM0XV8xxIWkU4mmvnBlR
Message ID: msg_nSfjXXUp9RmCwkGhdnbVKoIx
Role: assistant
Content: Today, April 9, 2025, in Seoul, South Korea, the weather is mildly warm with a high of 62°F (about 16°C) and a low of 50°F (about 10°C). There is a slight chance of rain with expected precipitation of about 0.01 inches【3:0†source】.
Created At: 1744198126
Metadata: {}
URL: https://www.weather25.com/asia/south-korea/seoul?page=month&month=April
Title: Seoul weather in April 2025 | Seoul 14 day weather
--------------------------------------------------
